<a href="https://colab.research.google.com/github/MutanteApps/chaotic_maps/blob/main/9D_chaotic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chaotic Map Iterator
Creates a class to see and work plots for caotic equations.

It just works with simple plots, with 1 to 3 variables or constants.

For use this, you need:
* A equation function, which only recieves a dict with the values of its constants and its variables, that also returns a dict with the new variables values.
* A dict with the initial values for the variables and constants, also 'iterations' that tells how many iterations to run in the initial state.



# Main Class

In [ ]:
# import to run the main class
import numpy as np # to create the arrays for the graphics
import matplotlib.pyplot as plt # to show information
from collections import defaultdict

# for tests and others
import pandas as pd
from itertools import combinations

In [ ]:
class Chaos:
    """
    This class creates a iterator like object of some cool function.
    For it to work you need to send a dict with the initial values, where each
    variable or constant name is the key and the value for it is the initial
    value for that. Also, you have to add the value for the number of iterations
    to the transient state, with the name 'iterations'
    """

    def __init__(self, values, equation=None):
        """
        This create the object with the initial values and the equation to
        iterate over.

        :param values: dict with the initial values for the equation
        :param equation: a function that takes a dictionary with the values for
        the calculation and returns a new dictionary with the next values
        """
        self.initial_values = values.copy()
        self.values = values.copy()
        self.iter_once = equation

    def iter(self, times=1)-> None:
        """
        Iterates over the equation to get new values.
        Changes in place and doesn't return anything.
        """
        for _ in range(int(times)):
            self.update_current_values(self.iter_once(**self.values))

    def change_initial_values(self, new_values:dict)->None:
        """
        Updates the values for the initial values of the dict for the
        new values passed in the dict.
        :param new_values: dict with the new_values
        """
        a = list(new_values.keys())
        b = list(self.initial_values.keys())
        if not (set(a+b) == set(b)):
            raise TypeError(f'Diferent Variables. \n    Expected: \n          {b}. \n    Got: \n          {a}.')
        self.initial_values = self.initial_values|new_values

    def update_current_values(self, values_dict):
        """
        Update the values for each equation
        """
        a = list(values_dict.keys())
        b = list(self.values.keys())
        assert set(a+b) == set(b), f'There are values outside the ones in the equation'

        self.values |= values_dict

    def create_array(self, size, new_values=None):
        """
        Create a list of numbers using the initial values updated by the values passed.
        """
        if new_values is None:
            new_values = self.initial_values.copy()

        self.init_chaos(new_values)
        values_array = defaultdict(list)

        for _ in range(int(size)):
            self.iter()
            for var, value in self.values.items():
                values_array[var].append(value)
        return values_array

    def init_chaos(self, new_values=None):
        """
        Initiates the equation with the values given or the initial values, then iter for given times,
        changing the values for each iteration
        :param new_values: None or dict with the new initial values
        """
        if new_values is None:
            new_values = self.initial_values
        else:
            a = list(new_values.keys())
            b = list(self.initial_values.keys())
            assert set(a+b) == set(b), f'Diferent Variables. \n    Expected: \n          {b}. \n    Got: \n          {a}.'

        self.values = self.initial_values|new_values

        self.iter(self.values.get('iterations', 1))

    def show_all_graphs_2d(self, variables = None, size = 100):
        if variables is None:
            variables = list(self.values.keys())
        vals = self.create_array(size)
        for x in variables:
            for y in variables:
                if x == y:
                    continue

                print("#"*50)

                X = vals.get(x)
                Y = vals.get(y)

                plt.plot(X, Y)
                plt.xlabel(x)
                plt.ylabel(y)
                plt.title(f'Values of {x} x {y}')
                plt.show()

    def show_histogram(self, variables=None, size=100, bins=10):
        vals = self.create_array(size)
        if variables is None:
            variables = list(self.values.keys())
        for x in variables:
            print("#"*50)
            plt.hist(vals.get(x), bins=bins)
            plt.title(x)
            plt.show()

    def show_graph(self, x, y=None, z=None, /, size=100,**kwargs):
        if z is None:
            self.show_graph_2d(x, y , size, **kwargs)
        else:
            self.show_graph_3d(x, y, z, size, **kwargs)

    def show_graph_2d(self, x:str, y:str, /, size=100, **kwargs):
        vals = self.create_array(size)


        if y is None:
            Y = vals.get(x)
            X = list(range(len(Y)))

            title = f'Values of {x.capitalize()} in time.'
            xlabel = '$n^{th}$ iteration'
            ylabel = x.capitalize()
        else:
            Y = vals.get(y)
            X = vals.get(x)
            title = f'Values of {x.capitalize()} x {y.capitalize()}'
            xlabel = x.capitalize()
            ylabel = y.capitalize()
        s = 5/(size/100)
        plt.scatter(X, Y, **{'s':s, 'color':'k'}|kwargs)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.title(title)
        plt.show()

    def show_graph_3d(self, x, y, z, /, size=100, **kwargs):
        """
        Plota um gráfico 3D das variáveis x, y e z ao longo das iterações.

        :param x: Nome da variável x.
        :param y: Nome da variável y.
        :param z: Nome da variável z.
        :param size: Número de iterações.
        :param kwargs: Argumentos adicionais para personalização do gráfico.
        """
        vals = self.create_array(size)

        X = vals.get(x)
        Y = vals.get(y)
        Z = vals.get(z)

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(X, Y, Z, **{'s': 5/(size/100), 'color': 'k'} | kwargs)

        ax.set_xlabel(x.capitalize())
        ax.set_ylabel(y.capitalize())
        ax.set_zlabel(z.capitalize())
        ax.set_title(f'Values of {x.capitalize()}, {y.capitalize()}, {z.capitalize()} over time')

        plt.show()


    def show_bifurcation_graph(self, x, initial_val, end_val, how_many, y, size, **kwargs):
        """
        :param x: str with the name for the x parameter
        :param initial_val: float for the initial value for parameter x
        :param end_val: float for the end value for parameter x
        :param how_many: int with steps from the initial and final values for x
        :param y: str with the name of the variable to find
        :param size: how many values of y to find
        """
        X = np.linspace(initial_val, end_val, int(how_many))
        xs = []
        ys = []
        for x_val in X:
            array_values = self.create_array(size = size, new_values={x:x_val})
            for value in array_values[y]:
                xs.append(x_val)
                ys.append(value)

        plt.scatter(xs, ys, **{'s':1e-5}|kwargs)
        plt.title(f'{x} x {y}')
        plt.xlabel(x)
        plt.ylabel(y)
        plt.show()

# Tent Map

## The Equation and initial values

In [ ]:
def tent_map(x,r,**kwargs):
    new_x = min([1-x, x])*r
    return {'x': new_x}

In [ ]:
initial_values = {'x': .3,'r': 1.9999999999, 'iterations':2e2}

In [ ]:
tent = Chaos(initial_values, tent_map)
tent.init_chaos()

In [ ]:
tent.show_graph('x')

## Bifurcation map

In [ ]:
tent.show_bifurcation_graph('r', 0.8, 2, 1e3, 'x', 1000, color='k', s=1e-5)

## The range of the values

In [ ]:
tent.show_histogram('x', bins=256, size=1e5)

## Graphs

In [ ]:
# create the most usual gaph for this system
array = tent.create_array(size=100)
xs = array.get('x')[1:]
ys = array.get('x')[0:-1]

plt.scatter(ys, xs)
plt.show()

# Logistic

## The Equation and initial values

In [ ]:
def logistic_map(x,r, **kwargs):
    new_x = r*x*(1-x)
    return {'x':new_x}

In [ ]:
initial_values = {'x': .3,'r': 3.94, 'iterations':1e2}

In [ ]:
logistic = Chaos(initial_values, logistic_map)
logistic.init_chaos()

## Bifurcation Map

In [ ]:
logistic.show_bifurcation_graph('r', 0, 4, 1000, 'x', 1000)

In [ ]:
logistic.show_bifurcation_graph('r', -2, 4, 1000, 'x', 1000)

## The values range

In [ ]:
logistic.show_histogram('x', bins=256, size=1000)

It is shown that the values tend to the highest values.

# Folded Towel Map

## The Equation and initial values

In [ ]:
def folded_towel(x,y,z,alpha,beta, **kwargs):

    new_x = alpha*x*(1-x)-.05*(y+.35)*(1-2*z)
    new_y = .1*((y+.35)*(1+2*z)-1)*(1-1.9*x)
    new_z = 3.78*z*(1-z)+beta*y

    new_dict = dict()
    new_dict['x'] = new_x
    new_dict['y'] = new_y
    new_dict['z'] = new_z

    return new_dict

In [ ]:
variables = {
    'x': 0.085,
    'y': -0.121,
    'z': 0.075,
    'alpha': 3.7,
    'beta': .1,
    'iterations': 400
}

In [ ]:
folded = Chaos(variables, folded_towel)
folded.init_chaos()

## Bifurcation Map

In [ ]:
folded.show_bifurcation_graph('alpha', 1, 3.9, 1000, 'x', size=1000)

## Graphs

In [ ]:
# usual graph for this map
folded.show_graph('y', 'x', size=100000)

In [ ]:
# the values frequency
folded.show_histogram('xyz')

# 9D Hyperchaos

## The Equation and initial values

Rayleigh-Bénard, from http://www.scholarpedia.org/article/Hyperchaos


In [ ]:
def chaotic_9D(c1, c2, c3, c4, c5, c6, c7, c8, c9, s, r, **kwargs):
        s_1_2 = 1+s**2

        b1 = 4*(s_1_2 / (1 + 2 * s ** 2))
        b2 = (1+2*s**2)/(2*s_1_2)
        b3 = 2*((1-s**2)/s_1_2)
        b4 = s ** 2 / s_1_2
        b5 = (8*s**2)/(1+2*s**2)
        b6 = 4/(1+2*s**2)

        limit_vals = lambda x: x%256

        new_c1 = limit_vals((-s*b1*c1 - c2*c4 + b4*c4**2 + b3*c3*c5 - s*b2*c7)+c1)
        new_c2 = limit_vals((-s*c2 + c1*c4 - c2*c5 + c4*c5 -s*c9/2)+c2)
        new_c3 = limit_vals((-s*b1*c3+ c2*c4 -b4*c2**2 - b3*c1*c5 + s*b2*c8)+c3)
        new_c4 = limit_vals((-s*c4 - c2*c3 - c2*c5 + c4*c5 + s*c9/2)+c4)
        new_c5 = limit_vals((-s*b5*c5 + c2**2/2 - c4**2/2)+c5)
        new_c6 = limit_vals((-b6*c6 + c2*c9 - c4*c9)+c6)
        new_c7 = limit_vals((-b1*c7 - r*c1 + 2*c5*c8 - c4*c9)+c7)
        new_c8 = limit_vals((-b1*c8 + r*c3 - 2*c5*c7 + c2*c9)+c8)
        new_c9 = limit_vals((-c9 - r*c2 + r*c4 - 2*c2*c6 + 2*c4*c6 + c4*c7 - c2*c8)+c9)


        new_dict = {
            "c1": new_c1,
            "c2": new_c2,
            "c3": new_c3,
            "c4": new_c4,
            "c5": new_c5,
            "c6": new_c6,
            "c7": new_c7,
            "c8": new_c8,
            "c9": new_c9,
        }
        return new_dict

In [ ]:
import random # to generate the initital values, change to control it.

variables = {}
for key in ["c1","c2","c3","c4","c5","c6","c7","c8","c9"]:
    variables[key]=random.random()

constants = {
        "s" : 5,
        "r"     : 43.3,
        'iterations': 1000
}

In [ ]:
nineD = Chaos(variables|constants, chaotic_9D)
nineD.init_chaos()

In [ ]:
nineD.show_graph('c1', 'c9', 'c5', size=10000)

In [ ]:
nineD.show_bifurcation_graph('r', -1e15, 1e15, 100, 'c2', 100, s=1)

In [ ]:
nineD.show_histogram([f'c{idx+1}' for idx in range(9)], bins=128, size=1e5)

## Test difference for close initial values

In [ ]:
next_numbers = 100
full_array = pd.DataFrame(nineD.create_array(next_numbers)) # the next 20 numbers generate by the system and the initial values

In [ ]:
full_array

In [ ]:
nineD2 = Chaos(variables|constants|{'s':5-1e-14}, chaotic_9D) # just changes the value of the 's' constant, by the minimum amount possible in pure python
full_array2 = pd.DataFrame(nineD2.create_array(next_numbers))
print(full_array2['s'][0]) # show the new valor for the new 's' constant

In [ ]:
diff_array = full_array-full_array2 # create a array with the diference from the values from nineD and nineD2

The initial values, even close to each other, create different arrays, with
different values and distances.
Below are the graphics with the differences for each iteration for each variable. Note that the possible for each variable range from 0 to 255, so
the variations are from -255 to 255.

In [ ]:
to_show = diff_array[[x for x in diff_array if x not in ['s', 'r', 'iterations']]] # avoid the variables and iterations

fig, axs = plt.subplots(3,3, figsize=(15, 15))

for ax, var in zip([item for sublist in axs for item in sublist],to_show):
    ax.set_title(var)
    ax.plot(to_show[var])

fig.suptitle('Diference from the values for each variable from nineD and nineD2')
plt.show()


## Bifurcation map for all variables.

In [ ]:
for constant in ['s', 'r']:
    for variable in ["c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9"]:
        nineD.show_bifurcation_graph(constant, 0, 1, 1000, variable, 100, s=.1)

## The range of the full_array with {`next_numbers`} numbers

In [ ]:
fig, axes = plt.subplots(3,3, figsize=(30,30))
for i in range(3):
    for j in range(3):
        axes[i,j].plot(full_array.get(f'c{i*3+j+1}'))
plt.show()

## Graphs with the values after the initial iterations.

### All 2D graphics

In [ ]:
cols=3
rows = len(list(combinations(["c1","c2","c3","c4","c5","c6","c7","c8","c9"], 3)))//cols

In [ ]:
rows

In [ ]:
nineD = Chaos(variables|constants, chaotic_9D)

for var_1, var_2 in combinations(["c1","c2","c3","c4","c5","c6","c7","c8","c9"], 2):
    nineD.show_graph(var_1, var_2, size=1000) # always uses the initial values

### All 3D graphics

In [ ]:
nineD = Chaos(variables|constants, chaotic_9D)
for var_1, var_2, var_3 in combinations(["c1","c2","c3","c4","c5","c6","c7","c8","c9"], 3):
    nineD.show_graph(var_1, var_2, var_3, size=1000) # always uses the initial values

## current and past values

In [ ]:
nineD = Chaos(variables|constants, chaotic_9D)
full_array = nineD.create_array(100)
for offset in range(1,15):
    for var in ["c1","c2","c3","c4","c5","c6","c7","c8","c9"]:
        Y = full_array[var][offset:] # next value
        X = full_array[var][:-offset] # previous value
        plt.plot(X, Y)
        plt.title(f'${var}$')
        plt.ylabel(f'Valor atual de ${var}$')
        plt.xlabel(f'Valor passado de ${offset}$ ${var}$')
        plt.show()

In [ ]:
nineD = Chaos(variables|constants, chaotic_9D)
full_array = nineD.create_array(100)
for offset in range(1,15):
    for var_1, var_2 in combinations(["c1","c2","c3","c4","c5","c6","c7","c8","c9"], 2):
        Y = full_array[var_2][offset:] # next value
        X = full_array[var_1][:-offset] # previous value
        plt.plot(X, Y)
        plt.title(f'${var_1}$ x ${var_2}$')
        plt.ylabel(f'Valor atual de ${var_2}$')
        plt.xlabel(f'Valor passado de ${offset}$ ${var_1}$')
        plt.show()

In [ ]:
nineD = Chaos(variables|constants, chaotic_9D)
full_array = nineD.create_array(10000)
for offset in range(1,15):
    for var_1, var_2 in combinations(["c1","c2","c3","c4","c5","c6","c7","c8","c9"], 2):
        Y = full_array[var_2][offset:] # next value
        X = full_array[var_1][:-offset] # previous value
        plt.scatter(X, Y, s=1e-1)
        plt.title(f'${var_1}$ x ${var_2}$')
        plt.ylabel(f'Valor atual de ${var_2}$')
        plt.xlabel(f'Valor passado de ${offset}$ ${var_1}$')
        plt.show()